# Open image and covert to dataframe

In [1]:
import spectral.io.envi as envi

# location for image 
loc = '/home/lql/Desktop/rs_mlsvm/data/' 

data1 = 'img_val'
data2 = 'mask_val'


img = envi.open(loc + data1 +'.hdr', loc + data1 + '.img')
img = img[:,:,:]
mask = envi.open(loc + data2 +'.hdr', loc + data2 + '.img')
mask = mask[:,:,:]

ModuleNotFoundError: No module named 'spectral'

In [122]:
print (mask.shape)
print (img.shape)

(160, 280, 1)
(160, 280, 260)


In [123]:
import numpy as np

# number of pixels per category (0 = no fire, 1 = fire, 2 = background)
(unique, counts) = np.unique(mask, return_counts=True)
np.asarray((unique, counts)).T

array([[    0, 42856],
       [    1,  1944]])

In [124]:
import pandas as pd
l = mask.flatten()
df = pd.DataFrame(data=l, columns=['label'])

In [125]:
# loop through bands and add to df
for i in range(0, img.shape[2]):
    #print (i)
    b = img[:,:,i].flatten()
    df['b'+str(i)] = b

In [8]:
# remove NaN (label = 2)
#df = df[df.label != 2]

In [126]:
df = df.dropna()

In [127]:
df['label'].value_counts()

0    42045
1     1887
Name: label, dtype: int64

# Need to change labels for MLSVM (-1,1)
### Make sure the number of minority class (lbl==1) is smaller than number of majority class (lbl==-1)

In [128]:
# no wildfire vs wildfire
df.label.value_counts()

0    42045
1     1887
Name: label, dtype: int64

In [129]:
df2 = df.copy()
df2 = df2.astype({'label': 'int16'})

In [130]:
df2['label'] = df2['label'].replace(0,-1)

In [131]:
df2.label.value_counts()

-1    42045
 1     1887
Name: label, dtype: int64

In [132]:
df2

,label,b0,b1,b2,b3,b4,b5,b6,b7,b8,...,b250,b251,b252,b253,b254,b255,b256,b257,b258,b259
0,-1,0.2824,0.3190,0.3285,0.3038,0.1717,0.0465,0.0239,0.060858,0.166470,...,0.1660,0.1711,0.2193,0.1884,0.1506,0.1176,0.0885,0.015129,0.024409,0.009139
1,-1,0.2824,0.3190,0.3285,0.3038,0.1717,0.0465,0.0239,0.060858,0.166470,...,0.1660,0.1711,0.2193,0.1884,0.1506,0.1176,0.0885,0.015129,0.024409,0.009139
2,-1,0.2305,0.2876,0.2701,0.2504,0.1617,0.0512,0.0246,0.110210,0.221369,...,0.4392,0.4420,0.4699,0.4593,0.3824,0.3705,0.2681,0.003177,0.012659,0.003041
3,-1,0.2305,0.2876,0.2701,0.2504,0.1617,0.0512,0.0246,0.110210,0.221369,...,0.4392,0.4420,0.4699,0.4593,0.3824,0.3705,0.2681,0.003177,0.012659,0.003041
4,-1,0.2118,0.2782,0.2427,0.2291,0.1559,0.0493,0.0224,0.135510,0.227787,...,0.1553,0.1949,0.1906,0.1768,0.1700,0.1300,0.0936,0.113078,0.141997,0.069866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44795,-1,0.0698,0.1620,0.0403,0.0641,0.1103,0.0615,0.0295,0.397757,0.180282,...,0.0275,0.1042,0.0255,0.0352,0.1251,0.0857,0.0404,0.582384,0.177884,0.182128
44796,-1,0.0762,0.1620,0.0485,0.0706,0.1042,0.0522,0.0249,0.360202,0.170855,...,0.0252,0.0835,0.0252,0.0325,0.0924,0.0649,0.0257,0.536339,0.139380,0.143667
44797,-1,0.2881,0.3304,0.2767,0.2907,0.1449,0.0638,0.0363,0.068391,0.107497,...,0.0977,0.1808,0.1067,0.1058,0.1541,0.1018,0.0516,0.298384,0.214895,0.160116
44798,-1,0.2285,0.2840,0.2127,0.2285,0.1294,0.0573,0.0294,0.108293,0.130927,...,0.0457,0.1409,0.0322,0.0451,0.1858,0.1306,0.0831,0.510182,0.202795,0.207981


In [133]:
df2.shape

(43932, 261)

In [47]:
# very imbalanced data
#from imblearn.datasets import make_imbalance

#x_imb, y_imb = make_imbalance(x, y, sampling_strategy={-1: 1000000, 1: 500}, random_state=0)

In [134]:
y = df2['label'].values
unique, counts = np.unique(y, return_counts=True)
print ('Training Counts')
print (np.asarray((unique, counts)).T)

Training Counts
[[   -1 42045]
 [    1  1887]]


In [135]:
loc = '/home/lql/Desktop/rs_mlsvm/repo/mlsvm/src/datasets/'
data3 = 'val_data.csv'
df2.to_csv(loc + data3,header=None, sep=',', index=None)
